In [1]:
import torch 
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

%cd ../

/workspace/Walk_Video_PyTorch/project


In [2]:
from  models.pytorchvideo_models import WalkVideoClassificationLightningModule

/usr/lib/python3.9/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [3]:
from main import get_parameters

opt, unknown = get_parameters()
opt.num_workers = 1

In [4]:
model = WalkVideoClassificationLightningModule(opt)
# model = model.load_from_checkpoint("/workspace/Walk_Video_PyTorch/logs/resnet/0603/checkpoints/epoch=99-step=1800.ckpt")
model.eval()

checkpoint = torch.load("/workspace/Walk_Video_PyTorch/logs/resnet/0603/checkpoints/epoch=99-step=1800.ckpt")
# hyper_parameters = checkpoint["hyper_parameters"]

In [5]:
from dataloader.data_loader import WalkDataModule

module = WalkDataModule(opt)
pred_data = module.predict_dataloader()

input_data = next(iter(pred_data))

In [6]:
input_data['video'].shape

input_data["video_name"]

['ca1.mp4',
 'ca2.mp4',
 'c3.mp4',
 'c2.mp4',
 'ca7.mp4',
 'c10.mp4',
 'ca6.mp4',
 'c9.mp4',
 'ca9.mp4',
 'c5.mp4',
 'c6.mp4',
 'ca5.mp4',
 'c4.mp4',
 'ca8.mp4',
 'ca4.mp4',
 'c7.mp4']

In [7]:
input_data['label']

tensor([1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0])

In [8]:
preds = model(input_data['video'])

In [9]:
post_act = torch.nn.Softmax(dim=1)
preds = post_act(preds)
pred_classes = preds.topk(k=1).indices

In [12]:
pred_classes

tensor([[1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1]])

In [13]:
classname = {}

classname[0] = 'asd'
classname[1] = 'lcs'

In [14]:
real_calss = []

for i in input_data['label'].tolist():
    real_calss.append(classname[i])

In [15]:
real_calss

['lcs',
 'lcs',
 'asd',
 'asd',
 'lcs',
 'asd',
 'lcs',
 'asd',
 'lcs',
 'asd',
 'asd',
 'lcs',
 'asd',
 'lcs',
 'lcs',
 'asd']

In [16]:
# pred_class_names = []
# for num in range(opt._BATCH_SIZE):
#     for i in pred_classes[i]:
#         pred_class_names.append(classname[int(i)])


pred_class_names = [classname[int(i)] for i in pred_classes]
print("Predicted labels: %s" % ", ".join(pred_class_names))
print("real label: %s" % ",".join(real_calss))

Predicted labels: lcs, lcs, lcs, lcs, lcs, lcs, lcs, lcs, lcs, lcs, lcs, lcs, lcs, lcs, lcs, lcs
real label: lcs,lcs,asd,asd,lcs,asd,lcs,asd,lcs,asd,asd,lcs,asd,lcs,lcs,asd


: 